In [ ]:
# import torch
# import torchvision
# import numpy as np
# import cv2
# from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
# from PIL import Image

# # Завантаження моделі Faster R-CNN з попередньо навченими вагами
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
# model.eval()

# # Використання CPU або CUDA (якщо доступно)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Функція для завантаження зображення
# def read_image(image_path):
#     image = Image.open(image_path).convert("RGB")
#     return np.array(image)

# # Функція для отримання передбачень об'єктів
# def get_prediction(image_path, model, confidence_threshold=0.5):
#     image = read_image(image_path)
#     image_tensor = torchvision.transforms.functional.to_tensor(image).unsqueeze(0).to(device)

#     # Отримання передбачень
#     with torch.no_grad():
#         predictions = model(image_tensor)[0]

#     # Фільтрація передбачень за порогом довіри
#     boxes = predictions['boxes'].cpu().numpy()
#     scores = predictions['scores'].cpu().numpy()
#     labels = predictions['labels'].cpu().numpy()

#     selected_indices = np.where(scores >= confidence_threshold)[0]
#     selected_boxes = boxes[selected_indices]
#     selected_labels = labels[selected_indices]
#     selected_scores = scores[selected_indices]

#     return selected_boxes, selected_labels, selected_scores

# # Функція для візуалізації результатів
# def visualize_predictions(image_path, boxes, labels, scores):
#     image = cv2.imread(image_path)
#     for box, label, score in zip(boxes, labels, scores):
#         x_min, y_min, x_max, y_max = map(int, box)
#         cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
#         label_text = f"Label: {label}, Score: {score:.2f}"
#         cv2.putText(image, label_text, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

#     # Збереження або показ зображення з передбаченнями
#     output_path = "prediction_visual1.png"
#     cv2.imwrite(output_path, image)
#     # cv2.imshow("predict", image)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
#     # Image(image)

# # Основний процес
# def main():
#     image_path = "image1.jpg"  # Заміни на свій шлях до зображення
#     boxes, labels, scores = get_prediction(image_path, model)
#     visualize_predictions(image_path, boxes, labels, scores)

# if __name__ == "__main__":
#     main()

In [ ]:
# import os
# import torch
# import torchvision
# from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
# from IPython.display import Image
# from PIL import Image as PILImage
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches

# os.getcwd()

# # Define AutoDetectionModel class
# class AutoDetectionModel:
#     def __init__(self, model, confidence_threshold, image_size, device, load_at_init):
#         self.model = model
#         self.confidence_threshold = confidence_threshold
#         self.image_size = image_size
#         self.device = device
#         self.load_at_init = load_at_init
#         self.model.to(self.device)
#         self.model.eval()

#     @classmethod
#     def from_pretrained(cls, model_type, model, confidence_threshold, image_size, device, load_at_init=True):
#         return cls(model, confidence_threshold, image_size, device, load_at_init)

#     def predict(self, image):
#         with torch.no_grad():
#             image = [img.to(self.device) for img in image]
#             predictions = self.model(image)
#             return predictions

# # Define get_sliced_prediction function
# def get_sliced_prediction(image_path, detection_model, slice_height, slice_width, overlap_height_ratio, overlap_width_ratio):
#     image = PILImage.open(image_path).convert("RGB")
#     image_np = np.array(image)
#     height, width, _ = image_np.shape

#     slice_height_overlap = int(slice_height * overlap_height_ratio)
#     slice_width_overlap = int(slice_width * overlap_width_ratio)

#     results = []
#     for y in range(0, height, slice_height - slice_height_overlap):
#         for x in range(0, width, slice_width - slice_width_overlap):
#             slice_x1 = x
#             slice_y1 = y
#             slice_x2 = min(x + slice_width, width)
#             slice_y2 = min(y + slice_height, height)

#             slice_image = image_np[slice_y1:slice_y2, slice_x1:slice_x2]
#             slice_image_tensor = torchvision.transforms.functional.to_tensor(slice_image)

#             predictions = detection_model.predict([slice_image_tensor])[0]
#             filtered_boxes = []
#             filtered_scores = []
#             for box, score, label in zip(predictions['boxes'], predictions['scores'], predictions['labels']):
#                 if score >= detection_model.confidence_threshold:
#                     filtered_boxes.append(box)
#                     filtered_scores.append(score)
#             predictions['boxes'] = filtered_boxes
#             predictions['scores'] = filtered_scores
#             results.append((predictions, (slice_x1, slice_y1, slice_x2, slice_y2)))

#     return SlicedPrediction(image_np, results)

# # Define SlicedPrediction class
# class SlicedPrediction:
#     def __init__(self, original_image, results):
#         self.original_image = original_image
#         self.results = results

#     def export_visuals(self, export_dir=""):
#         if not os.path.exists(export_dir) and export_dir != "":
#             os.makedirs(export_dir)
#         fig, ax = plt.subplots(1, figsize=(12, 9))
#         ax.imshow(self.original_image)
#         for predictions, (x1, y1, x2, y2) in self.results:
#             for box, score in zip(predictions['boxes'], predictions['scores']):
#                 if score >= 0.7:  # Adjust confidence threshold for visualization
#                     rect = patches.Rectangle((x1 + box[0], y1 + box[1]), box[2] - box[0], box[3] - box[1], linewidth=2, edgecolor='red', facecolor='none')
#                     ax.add_patch(rect)
#         plt.axis('off')
#         output_path = f"{export_dir}/prediction_visual.png"
#         plt.savefig(output_path)
#         plt.close(fig)
#         return output_path

# # set torchvision FasterRCNN model
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# # initialize detection model using AutoDetectionModel
# detection_model = AutoDetectionModel.from_pretrained(
#     model_type='torchvision',
#     model=model,
#     confidence_threshold=0.5,
#     image_size=640,
#     device="cpu", # or "cuda:0"
#     load_at_init=True,
# )

# # perform prediction using sliced inference
# result = get_sliced_prediction(
#     "image1.jpg",
#     detection_model,
#     slice_height=320,
#     slice_width=320,
#     overlap_height_ratio=0.2,
#     overlap_width_ratio=0.2,
# )

# # export the prediction visuals
# output_path = result.export_visuals(export_dir="")

# # display the result image
# Image(output_path)


In [ ]:
!pip install sahi

In [ ]:
!pip install ultralytics

In [ ]:
!pip install yolov5

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("using", device, "device")

In [ ]:
import cv2
import numpy as np
from sahi.utils.yolov5 import download_yolov5s6_model
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_sliced_prediction
import torch
import os
# from google.colab.patches import cv2_imshow
# Install the required packages
# try:
#     import yolov5
# except ModuleNotFoundError:
#     print("YOLOv5 is not installed. Installing now...")
#     !pip install -U torch sahi yolov5

# Download YOLOv5 model
model_path = 'models/yolov5s6.pt'
download_yolov5s6_model(destination_path=model_path)

# Initialize the detection model
confidence_threshold = 0.3
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=model_path,
    confidence_threshold=confidence_threshold,
    device="cuda"
)

# Load the video file
video_path = "/content/Cars Moving On Road Stock Footage - Free Download.mp4"  # Replace with the path to your video
cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

# Loop through the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if there are no frames left to read

    # Get sliced prediction for the current frame
    prediction = get_sliced_prediction(frame, detection_model, slice_height=256, slice_width=256, overlap_height_ratio=0.2, overlap_width_ratio=0.2)

    # Draw the detections on the frame
    for obj in prediction.object_prediction_list:
        if obj.category.name == "car":  # Filter for cars
            bbox = obj.bbox.to_xyxy()
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{obj.category.name}: {obj.score:.2f}" if isinstance(obj.score, float) else f"{obj.category.name}: {obj.score}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with detections
    cv2.imshow(frame)

    # Press 'q' to exit the video early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from typing import Dict, List, Optional, Sequence, Tuple, Union
import numpy as np
from PIL import Image
from ultralytics import YOLO
import time
import matplotlib.pyplot as plt


def get_slice_bboxes(
    image_height: int,
    image_width: int,
    slice_height: Optional[int] = None,
    slice_width: Optional[int] = None,
    auto_slice_resolution: bool = True,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
) -> List[List[int]]:

    slice_bboxes = []
    y_max = y_min = 0

    if slice_height and slice_width:
        y_overlap = int(overlap_height_ratio * slice_height)
        x_overlap = int(overlap_width_ratio * slice_width)
    else:
        raise ValueError("Compute type is not auto and slice width and height are not provided.")

    while y_max < image_height:
        x_min = x_max = 0
        y_max = y_min + slice_height
        while x_max < image_width:
            x_max = x_min + slice_width
            if y_max > image_height or x_max > image_width:
                xmax = min(image_width, x_max)
                ymax = min(image_height, y_max)
                xmin = max(0, xmax - slice_width)
                ymin = max(0, ymax - slice_height)
                slice_bboxes.append([xmin, ymin, xmax, ymax])
            else:
                slice_bboxes.append([x_min, y_min, x_max, y_max])
            x_min = x_max - x_overlap
        y_min = y_max - y_overlap
    return slice_bboxes


class SlicedImage:
    def __init__(self, image, starting_pixel):
        self.image = image
        self.starting_pixel = starting_pixel


class SliceImageResult:
    def __init__(self, original_image_size: List[int], image_dir: Optional[str] = None):
        self.original_image_height = original_image_size[0]
        self.original_image_width = original_image_size[1]
        self.image_dir = image_dir

        self._sliced_image_list: List[SlicedImage] = []

    def add_sliced_image(self, sliced_image: SlicedImage):
        if not isinstance(sliced_image, SlicedImage):
            raise TypeError("sliced_image must be a SlicedImage instance")

        self._sliced_image_list.append(sliced_image)

    @property
    def sliced_image_list(self):
        return self._sliced_image_list

    @property
    def images(self):
        images = []
        for sliced_image in self._sliced_image_list:
            images.append(sliced_image.image)
        return images

    @property
    def starting_pixels(self) -> List[int]:
        starting_pixels = []
        for sliced_image in self._sliced_image_list:
            starting_pixels.append(sliced_image.starting_pixel)
        return starting_pixels

    @property
    def filenames(self) -> List[int]:
        filenames = []
        for sliced_image in self._sliced_image_list:
            filenames.append(sliced_image.coco_image.file_name)
        return filenames

    def __getitem__(self, i):
        def _prepare_ith_dict(i):
            return {
                "image": self.images[i],
                "starting_pixel": self.starting_pixels[i],
            }

        if isinstance(i, np.ndarray):
            i = i.tolist()

        if isinstance(i, int):
            return _prepare_ith_dict(i)
        elif isinstance(i, slice):
            start, stop, step = i.indices(len(self))
            return [_prepare_ith_dict(i) for i in range(start, stop, step)]
        elif isinstance(i, (tuple, list)):
            accessed_mapping = map(_prepare_ith_dict, i)
            return list(accessed_mapping)
        else:
            raise NotImplementedError(f"{type(i)}")

    def __len__(self):
        return len(self._sliced_image_list)


def slice_image(
    image: Union[str, Image.Image],
    slice_height: Optional[int] = None,
    slice_width: Optional[int] = None,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
    auto_slice_resolution: bool = True,
    min_area_ratio: float = 0.1,
    out_ext: Optional[str] = None,
    verbose: bool = False,
) -> SliceImageResult:

    image_pil = image

    image_width, image_height = image_pil.size
    if not (image_width != 0 and image_height != 0):
        raise RuntimeError(f"invalid image size: {image_pil.size} for 'slice_image'.")
    slice_bboxes = get_slice_bboxes(
        image_height=image_height,
        image_width=image_width,
        auto_slice_resolution=auto_slice_resolution,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=overlap_height_ratio,
        overlap_width_ratio=overlap_width_ratio,
    )

    n_ims = 0

    sliced_image_result = SliceImageResult(original_image_size=[image_height, image_width])

    image_pil_arr = np.asarray(image_pil)

    for slice_bbox in slice_bboxes:
        n_ims += 1

        tlx = slice_bbox[0]
        tly = slice_bbox[1]
        brx = slice_bbox[2]
        bry = slice_bbox[3]
        image_pil_slice = image_pil_arr[tly:bry, tlx:brx]

        slice_width = slice_bbox[2] - slice_bbox[0]
        slice_height = slice_bbox[3] - slice_bbox[1]

        sliced_image = SlicedImage(
            image=image_pil_slice, starting_pixel=[slice_bbox[0], slice_bbox[1]]
        )
        sliced_image_result.add_sliced_image(sliced_image)

    return sliced_image_result


cap = cv2.VideoCapture("D:\magl\code_jet\Cars1.mp4") #"road.mp4" "Cars1.mp4"

model = YOLO("yolov8s.pt")


# Get the width and height of the video frame
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))



while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    start_time = time.perf_counter()

    image = Image.fromarray(frame)
    height, width = image.size

    slice_height = int(height/4)
    slice_width = int(width/4)

    slice_image_result = slice_image(
            image=image,
            slice_height=slice_height,
            slice_width=slice_width,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2,
            auto_slice_resolution=False,
        )

    number_of_tiles = len(slice_image_result)

    results = []
    bboxes = []
    confs = []
    class_ids = []

    for i, image_slice in enumerate(slice_image_result):

        window = image_slice['image']
        start_x, start_y = image_slice['starting_pixel']

        results = model.predict(window, conf=0.2)

        for result in results:

            boxes = result.boxes  # Boxes object for bounding box outputs

            xyxy = boxes.xyxy.cpu().numpy()

            if xyxy.size == 0:
                continue

            xyxy = xyxy
            conf = boxes.conf.cpu().numpy()
            class_id = boxes.cls.cpu().numpy()

            for i in range(len(xyxy)):

                x1, y1, x2, y2 = xyxy[i]

                x1 += start_x
                y1 += start_y
                x2 += start_x
                y2 += start_y

                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                bboxes.append([x1, y1, x2, y2])
                confs.append(conf[i])
                class_ids.append(class_id[i])



    end_time = time.perf_counter()
    total_time = end_time - start_time
    fps = 1 / total_time

    for i in range(len(bboxes)):
        x1, y1, x2, y2 = bboxes[i]

        conf = confs[i]
        class_id = class_ids[i]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'{model.names[int(class_id)]} {conf:.1f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'{number_of_tiles}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'{fps:.1f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    # Write the frame into the file 'output.mp4'
    out.write(frame)

    cv2.imshow('frame', frame)

    # cv2.imshow('frame', frame)
    # #cv2_imshow(frame)
    # cv2.waitKey(0)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from typing import Dict, List, Optional, Sequence, Tuple, Union
import numpy as np
from PIL import Image
from ultralytics import YOLO
import time
import torch
import torchvision.transforms as T


def get_slice_bboxes(
    image_height: int,
    image_width: int,
    slice_height: Optional[int] = None,
    slice_width: Optional[int] = None,
    auto_slice_resolution: bool = True,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
) -> List[List[int]]:

    slice_bboxes = []
    y_max = y_min = 0

    if slice_height and slice_width:
        y_overlap = int(overlap_height_ratio * slice_height)
        x_overlap = int(overlap_width_ratio * slice_width)
    else:
        raise ValueError("Compute type is not auto and slice width and height are not provided.")

    while y_max < image_height:
        x_min = x_max = 0
        y_max = y_min + slice_height
        while x_max < image_width:
            x_max = x_min + slice_width
            if y_max > image_height or x_max > image_width:
                xmax = min(image_width, x_max)
                ymax = min(image_height, y_max)
                xmin = max(0, xmax - slice_width)
                ymin = max(0, ymax - slice_height)
                slice_bboxes.append([xmin, ymin, xmax, ymax])
            else:
                slice_bboxes.append([x_min, y_min, x_max, y_max])
            x_min = x_max - x_overlap
        y_min = y_max - y_overlap
    return slice_bboxes


class SlicedImage:
    def __init__(self, image, starting_pixel):
        self.image = image
        self.starting_pixel = starting_pixel


class SliceImageResult:
    def __init__(self, original_image_size: List[int], image_dir: Optional[str] = None):
        self.original_image_height = original_image_size[0]
        self.original_image_width = original_image_size[1]
        self.image_dir = image_dir

        self._sliced_image_list: List[SlicedImage] = []

    def add_sliced_image(self, sliced_image: SlicedImage):
        if not isinstance(sliced_image, SlicedImage):
            raise TypeError("sliced_image must be a SlicedImage instance")

        self._sliced_image_list.append(sliced_image)

    @property
    def sliced_image_list(self):
        return self._sliced_image_list

    @property
    def images(self):
        images = []
        for sliced_image in self._sliced_image_list:
            images.append(sliced_image.image)
        return images

    @property
    def starting_pixels(self) -> List[int]:
        starting_pixels = []
        for sliced_image in self._sliced_image_list:
            starting_pixels.append(sliced_image.starting_pixel)
        return starting_pixels

    def __getitem__(self, i):
        def _prepare_ith_dict(i):
            return {
                "image": self.images[i],
                "starting_pixel": self.starting_pixels[i],
            }

        if isinstance(i, np.ndarray):
            i = i.tolist()

        if isinstance(i, int):
            return _prepare_ith_dict(i)
        elif isinstance(i, slice):
            start, stop, step = i.indices(len(self))
            return [_prepare_ith_dict(i) for i in range(start, stop, step)]
        elif isinstance(i, (tuple, list)):
            accessed_mapping = map(_prepare_ith_dict, i)
            return list(accessed_mapping)
        else:
            raise NotImplementedError(f"{type(i)}")

    def __len__(self):
        return len(self._sliced_image_list)


def slice_image(
    image: Union[str, Image.Image],
    slice_height: Optional[int] = None,
    slice_width: Optional[int] = None,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2,
    auto_slice_resolution: bool = True,
    min_area_ratio: float = 0.1,
    out_ext: Optional[str] = None,
    verbose: bool = False,
) -> SliceImageResult:

    image_pil = image

    image_width, image_height = image_pil.size
    if not (image_width != 0 and image_height != 0):
        raise RuntimeError(f"invalid image size: {image_pil.size} for 'slice_image'.")
    slice_bboxes = get_slice_bboxes(
        image_height=image_height,
        image_width=image_width,
        auto_slice_resolution=auto_slice_resolution,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=overlap_height_ratio,
        overlap_width_ratio=overlap_width_ratio,
    )

    n_ims = 0

    sliced_image_result = SliceImageResult(original_image_size=[image_height, image_width])

    image_pil_arr = np.asarray(image_pil)

    for slice_bbox in slice_bboxes:
        n_ims += 1

        tlx = slice_bbox[0]
        tly = slice_bbox[1]
        brx = slice_bbox[2]
        bry = slice_bbox[3]
        image_pil_slice = image_pil_arr[tly:bry, tlx:brx]

        sliced_image = SlicedImage(
            image=image_pil_slice, starting_pixel=[slice_bbox[0], slice_bbox[1]]
        )
        sliced_image_result.add_sliced_image(sliced_image)

    return sliced_image_result


cap = cv2.VideoCapture("D:\magl\code_jet\Cars1.mp4")

model = YOLO("D:\magl\code_jet\last.pt")

# Check if CUDA is available and set the device to GPU if possible
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Get the width and height of the video frame
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

# Define transformation to convert image slice to the required BCHW format
transform = T.Compose([
    T.ToTensor(),
    T.Resize((640, 640)),  # Resize to model input size
])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.perf_counter()

    # Remove CUDA GpuMat usage since OpenCV might not have CUDA support
    # Directly use the frame for processing
    image = Image.fromarray(frame)
    height, width = image.size

    slice_height = int(height / 1)
    slice_width = int(width / 1)

    slice_image_result = slice_image(
        image=image,
        slice_height=slice_height,
        slice_width=slice_width,
        overlap_height_ratio=0.9, #0.2
        overlap_width_ratio=0.9,
        auto_slice_resolution=False,
    )

    number_of_tiles = len(slice_image_result)

    bboxes = []
    confs = []
    class_ids = []

    for i, image_slice in enumerate(slice_image_result):
        window = image_slice['image']
        start_x, start_y = image_slice['starting_pixel']

        # Convert the window to tensor and resize to match model input requirements
        window_tensor = transform(Image.fromarray(window)).unsqueeze(0).to(device)

        # Run inference
        results = model.predict(window_tensor, conf=0.2, device=device)

        for result in results:
            boxes = result.boxes  # Boxes object for bounding box outputs
            xyxy = boxes.xyxy.cpu().numpy()

            if xyxy.size == 0:
                continue

            conf = boxes.conf.cpu().numpy()
            class_id = boxes.cls.cpu().numpy()

            for i in range(len(xyxy)):
                x1, y1, x2, y2 = xyxy[i]

                x1 += start_x
                y1 += start_y
                x2 += start_x
                y2 += start_y

                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                bboxes.append([x1, y1, x2, y2])
                confs.append(conf[i])
                class_ids.append(class_id[i])

    end_time = time.perf_counter()
    total_time = end_time - start_time
    fps = 1 / total_time

    for i in range(len(bboxes)):
        x1, y1, x2, y2 = bboxes[i]

        conf = confs[i]
        class_id = class_ids[i]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'{model.names[int(class_id)]} {conf:.1f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'{number_of_tiles}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'{fps:.1f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the frame into the file 'output.mp4'
    out.write(frame)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
from ultralytics.models.yolo.detect.predict import DetectionPredictor
import cv2

model = YOLO("D:\magl\code_jet\yolov8s.pt")

results = model.predict(source="D:\magl\code_jet\IMG_9014.MP4", show=True)

#print(result)
cv2.destroyAllWindows()

In [ ]:
# Ultralytics YOLO 🚀, AGPL-3.0 license

import argparse
from pathlib import Path

import cv2
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.yolov8 import download_yolov8s_model

from ultralytics.utils.files import increment_path
from ultralytics.utils.plotting import Annotator, colors


class SAHIInference:
    """Runs YOLOv8 and SAHI for object detection on video with options to view, save, and track results."""

    def __init__(self):
        """Initializes the SAHIInference class for performing sliced inference using SAHI with YOLOv8 models."""
        self.detection_model = None

    def load_model(self, weights):
        """Loads a YOLOv8 model with specified weights for object detection using SAHI."""
        yolov8_model_path = f"models/{weights}"
        download_yolov8s_model(yolov8_model_path)
        self.detection_model = AutoDetectionModel.from_pretrained(
            model_type="yolov8", model_path=yolov8_model_path, confidence_threshold=0.3, device="cuda"
        )

    def inference(
        self, weights="yolov8n.pt", source="test.mp4", view_img=False, save_img=False, exist_ok=False, track=False
    ):
        """
        Run object detection on a video using YOLOv8 and SAHI.

        Args:
            weights (str): Model weights path.
            source (str): Video file path.
            view_img (bool): Show results.
            save_img (bool): Save results.
            exist_ok (bool): Overwrite existing files.
            track (bool): Enable object tracking with SAHI
        """
        # Video setup
        cap = cv2.VideoCapture(source)
        assert cap.isOpened(), "Error reading video file"
        frame_width, frame_height = int(cap.get(3)), int(cap.get(4))

        # Output setup
        save_dir = increment_path(Path("ultralytics_results_with_sahi") / "exp", exist_ok)
        save_dir.mkdir(parents=True, exist_ok=True)
        video_writer = cv2.VideoWriter(
            str(save_dir / f"{Path(source).stem}.mp4"),
            cv2.VideoWriter_fourcc(*"mp4v"),
            int(cap.get(5)),
            (frame_width, frame_height),
        )

        # Load model
        self.load_model(weights)
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            annotator = Annotator(frame)  # Initialize annotator for plotting detection and tracking results
            results = get_sliced_prediction(
                frame,
                self.detection_model,
                slice_height=512,
                slice_width=512,
                overlap_height_ratio=0.2,
                overlap_width_ratio=0.2,
            )
            detection_data = [
                (det.category.name, det.category.id, (det.bbox.minx, det.bbox.miny, det.bbox.maxx, det.bbox.maxy))
                for det in results.object_prediction_list
            ]

            for det in detection_data:
                annotator.box_label(det[2], label=str(det[0]), color=colors(int(det[1]), True))

            if view_img:
                cv2.imshow(Path(source).stem, frame)
            if save_img:
                video_writer.write(frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        video_writer.release()
        cap.release()
        cv2.destroyAllWindows()

    def parse_opt(self):
        """Parse command line arguments."""
        parser = argparse.ArgumentParser()
        parser.add_argument("--weights", type=str, default="yolov8n.pt", help="initial weights path")
        parser.add_argument("--source", type=str, required=True, help="video file path")
        parser.add_argument("--view-img", action="store_true", help="show results")
        parser.add_argument("--save-img", action="store_true", help="save results")
        parser.add_argument("--exist-ok", action="store_true", help="existing project/name ok, do not increment")
        return parser.parse_args()


if __name__ == "__main__":
    inference = SAHIInference()
    # inference.inference(**vars(inference.parse_opt()))
    inference.inference(source="D:\magl\code_jet\Cars1.mp4", view_img=True, save_img=True)

In [ ]:
from ultralytics import YOLO

# Load a model

model = YOLO("D:\magl\code_jet\last.pt")  # load a custom trained model

# Export the model
model.export(format="onnx")
# engine or onnx

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.export(format="engine", batch=8, workspace=4, int8=True, data="coco.yaml")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Завантаження YOLOv8 ваг та конфігураційного файлу
from ultralytics import YOLO

# Load the exported ONNX model
onnx_model = YOLO("yolov8s.onnx")

# Run inference
results = onnx_model("D:\magl\code_jet\image1.jpg")

# Обробка та відображення результатів
results = results[0]  # Отримуємо перший результат із списку
annotated_image = results.plot()  # Відображення результатів на зображенні

# Відображення зображення з результатами за допомогою Matplotlib
plt.imshow(annotated_image)
plt.axis('off')
plt.show()